In [14]:
yoi = [2020]
toi = ['15RTN', '15RTP', '14RNS']
root_path = "C:/Users/mcecil/CGA/CDL/"
spath = root_path + f"CDL_HLS_dataframe{yoi[0]}.csv"
url_file = root_path + f'CDL_HLS_dataframe{yoi[0]}.csv'
index_file = root_path + f'mask_index_{yoi[0]}.csv'
kml_file = root_path + 'sentinel_tile_grid.kml'
geojson_file = root_path + 'aoi.geojson'  ## chip file, lat-long
geojson_rpj_file = root_path + 'aoi_rpj.geojson'
hdf_dir = root_path + 'hdf/'
tiff_dir = root_path + 'tif/'
mask_dir = root_path + 'mask/'

tile_src_path = root_path + "tile_src.csv"

In [15]:

# Switch plotting backend
import matplotlib
matplotlib.use('agg')

# Import required modules
#import cartopy.crs as ccrs
import os
from datetime import datetime
import pandas as pd
import geopandas
#import pyhdf
import rasterio
import rasterio.mask
import matplotlib.pyplot as pp
import nasa_hls
import numpy as np
import time
import fiona
from pathlib import Path
import time
import urllib.request as urlreq
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [16]:
# initialize df for storing QA info
chip_image_df = pd.DataFrame(columns = ['chip_id', 'tile', 'minx', 'miny', 'maxx', 'maxy', 'image_id', 'date', 'month', 'ncell', 'QA_score'])

In [17]:
# Load the HLS tiles and place there coordinates into a numpy array for processing later
fiona.drvsupport.supported_drivers['KML'] = 'rw'
tile_src = geopandas.read_file(kml_file, driver='KML')
tile_name = []
tile_x = []
tile_y = []
for tile_ind in range(tile_src.shape[0]):
    tile_name.append(tile_src.iloc[tile_ind].Name)
    tile_x.append(tile_src.iloc[tile_ind].geometry.centroid.x)
    tile_y.append(tile_src.iloc[tile_ind].geometry.centroid.y)
tile_name = np.array(tile_name)
tile_x = np.array(tile_x)
tile_y = np.array(tile_y)
tile_src = pd.concat([tile_src, tile_src.bounds], axis = 1)
#del tile_src
tile_src.head(5)

,Name,Description,geometry,minx,miny,maxx,maxy
0,01CCV,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -7...,-180.0,-73.064633,180.0,-72.012478
1,01CDH,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-83.835334,180.0,-82.796720
2,01CDJ,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-82.939452,180.0,-81.906947
3,01CDK,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-82.044055,180.0,-81.016439
4,01CDL,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-81.148070,180.0,-80.124456


In [18]:
# Load the HLS query csv for this year
qfn = open(url_file)
qtile = []
qyear = []
qmonth = []
qday = []
qdate = []
qurl = []
for i, line in enumerate(qfn):
    if (i == 0): # Skip header
        continue
    dummy = line.split(",")
    qtile.append(dummy[2])
    qdate.append(dummy[3])
    qurl.append(dummy[4])
    
    qday.append(int(dummy[3].split("-")[2]))
    qmonth.append(int(dummy[3].split("-")[1]))
    qyear.append(int(dummy[3].split("-")[0]))
qfn.close()
qdate = np.array(qdate)
qtile = np.array(qtile)
qurl = np.array(qurl)
qmonth = np.array(qmonth)
qyear = np.array(qyear)

In [19]:
# Create the index file
# fn_index = open(index_file, "w")
# fn_index.write('EventID,Name,FireDate,MaskDate,BurnAcres,Tile,MaskFile')
# fn_index.close()

Load geojson used for query. 

In [20]:
aoi_src = geopandas.read_file(geojson_file) ## this could be a list
nfeatures = aoi_src.shape[0]
aoi_src = pd.concat([aoi_src, aoi_src.bounds], axis = 1)
aoi_src['centroid_x'] = aoi_src.centroid.x
aoi_src['centroid_y'] = aoi_src.centroid.y
aoi_src.head(5)

C:\Users\mcecil\AppData\Local\Temp\ipykernel_5544\513036168.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoi_src['centroid_x'] = aoi_src.centroid.x
C:\Users\mcecil\AppData\Local\Temp\ipykernel_5544\513036168.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoi_src['centroid_y'] = aoi_src.centroid.y


,geometry,minx,miny,maxx,maxy,centroid_x,centroid_y
0,"POLYGON ((-98.46590 28.48216, -98.46590 28.491...",-98.492844,28.482162,-98.465904,28.491892,-98.479374,28.487027


Loop through aoi_src (chips), associate closest tile, add ID field, download hdf (WORKING)

In [ ]:
for aoi_ind in range(nfeatures):
    aoi_row = aoi_src.iloc[aoi_ind]
    print(aoi_row)
    print(aoi_src.bounds)
    aoi_x = float(aoi_row.centroid_x)
    aoi_y = float(aoi_row.centroid_y)
        
    # Identify what tile the burn scar is in
    s = (tile_x-aoi_x)**2+(tile_y-aoi_y)**2
    tname = tile_name[np.argmin(s)]
    print(s)
    print(tname)
    aoi_src.at[aoi_ind, 'tile'] = tname
    aoi_src.at[aoi_ind, 'chip_id'] = 'chip_' + str(aoi_ind)
    
    
    # Subset potential images based on tile and date
    tile_mask = qtile == tname
    # Subset based on date
    date_mask1 = (qyear == yoi[0])
  #  diff = (qmonth-aoi_month)%12
   # date_mask2 = ((diff <= month_threshold[1]) & (diff >= month_threshold[0]))
    date_mask = date_mask1  #& date_mask2
    
    mask = tile_mask & date_mask
    candidate_dates = qdate[mask]
    candidate_tiles = qtile[mask]
    candidate_urls = qurl[mask]
    # Loop through images and check cloud cover
    for cd, ct, cu in zip(candidate_dates, candidate_tiles, candidate_urls):     
        local_name = cu.split('/')[-1].replace("\n", "")
        # Download a candidate file to check cloud cover
        try:
            urlreq.urlretrieve(cu, filename=hdf_dir+local_name)
        except:
            continue
#aoi_src.head(5)

Extract cloud cover and spatial coverage from metadata (NOT WORKING)

In [ ]:
for aoi_ind in range(nfeatures):
    aoi_row = aoi_src.iloc[aoi_ind]
    print(aoi_row)
    print(aoi_src.bounds)
    aoi_x = float(aoi_row.centroid_x)
    aoi_y = float(aoi_row.centroid_y)
        
    # Identify what tile the burn scar is in
    s = (tile_x-aoi_x)**2+(tile_y-aoi_y)**2
    tname = tile_name[np.argmin(s)]
    print(s)
    print(tname)
    aoi_src.at[aoi_ind, 'tile'] = tname
    aoi_src.at[aoi_ind, 'chip_id'] = 'chip_' + str(aoi_ind)
    
    # Subset potential images based on tile and date
    tile_mask = qtile == tname
    # Subset based on date
    date_mask1 = (qyear == yoi[0])
  #  diff = (qmonth-aoi_month)%12
   # date_mask2 = ((diff <= month_threshold[1]) & (diff >= month_threshold[0]))
    date_mask = date_mask1  #& date_mask2
    
    mask = tile_mask & date_mask
    candidate_dates = qdate[mask]
    candidate_tiles = qtile[mask]
    candidate_urls = qurl[mask]
    # Loop through images and check cloud cover
    for cd, ct, cu in zip(candidate_dates, candidate_tiles, candidate_urls):
       # print(cu)
        local_name = cu.split('/')[-1].replace("\n", "")
        
        if local_name != "HLS.S30.T14RNS.2020117.v1.4.hdf":
            continue
        # Download a candidate file to check cloud cover
        try:
            md = nasa_hls.get_metadata_from_hdf(hdf_dir+local_name, fields=['cloud_cover', 'spatial_coverage'])
            print(md)
       #     if ((md["cloud_cover"] < cldfrac_threshold) and (md['spatial_coverage'] > spacecov_threshold)):
        except:
            continue
        
aoi_src.head(5)

Convert HDF to single-layer TIF (Using HLS library, DOES NOT WORK)

In [ ]:
for aoi_ind in range(nfeatures):
    aoi_row = aoi_src.iloc[aoi_ind]
    print(aoi_row)
    print(aoi_src.bounds)
    aoi_x = float(aoi_row.centroid_x)
    aoi_y = float(aoi_row.centroid_y)
        
    # Identify what tile the burn scar is in
    s = (tile_x-aoi_x)**2+(tile_y-aoi_y)**2
    tname = tile_name[np.argmin(s)]
    print(s)
    print(tname)
    aoi_src.at[aoi_ind, 'tile'] = tname
    aoi_src.at[aoi_ind, 'chip_id'] = 'chip_' + str(aoi_ind)
    
    # Subset potential images based on tile and date
    tile_mask = qtile == tname
    # Subset based on date
    date_mask1 = (qyear == yoi[0])
  #  diff = (qmonth-aoi_month)%12
   # date_mask2 = ((diff <= month_threshold[1]) & (diff >= month_threshold[0]))
    date_mask = date_mask1  #& date_mask2
    
    mask = tile_mask & date_mask
    candidate_dates = qdate[mask]
    candidate_tiles = qtile[mask]
    candidate_urls = qurl[mask]
    # Loop through images and check cloud cover
    for cd, ct, cu in zip(candidate_dates, candidate_tiles, candidate_urls):
        print(cu)
        local_name = cu.split('/')[-1].replace("\n", "")
        
        if local_name != "HLS.S30.T14RNS.2020117.v1.4.hdf":
            continue
        
        nasa_hls.convert_hdf2tiffs(Path(hdf_dir+local_name), Path(tiff_dir))

Convert HDF to single-layer TIF (Mike's way - works)

In [ ]:
import osgeo
from osgeo import gdal
import glob
from subprocess import Popen, PIPE

bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B10', 'B11', 'B12', 'QA']


hdf_files = glob.glob(hdf_dir + "*.hdf")
for hdf_file in hdf_files:
    print(hdf_file)
    file_name = hdf_file.split('\\')[-1].replace('.hdf', '')
    if file_name != "HLS.S30.T14RNS.2020117.v1.4":
        continue
    print('valid')
    for long_band_name in bands:
        cmd = f'gdal_translate HDF4_EOS:EOS_GRID:"' + hdf_file + '":Grid:' + long_band_name + ' ' + tiff_dir + file_name + '__' + long_band_name + '.tif'
      #  print(cmd)
        p = Popen(cmd, stdout=PIPE, shell=True)
       # print(p)
        output, err = p.communicate()
        #print(output)
        #print(err)
  #  nasa_hls.convert_hdf2tif_dirfs(Path(hdf_file), Path(tif_dir_dir))
    # ds = gdal.Open(hdf_file)
    # ds_t = gdal.Translate('output.tif', hdf_file)
#ds = None
#print(ds)


In [36]:
for aoi_ind in range(nfeatures):
    aoi_row = aoi_src.iloc[aoi_ind]
    print(aoi_row)
    print(aoi_src.bounds)
    aoi_x = float(aoi_row.centroid_x)
    aoi_y = float(aoi_row.centroid_y)
        
    # Identify what tile the burn scar is in
    s = (tile_x-aoi_x)**2+(tile_y-aoi_y)**2
    tname = tile_name[np.argmin(s)]
    print(s)
    print(tname)
    aoi_src.at[aoi_ind, 'tile'] = tname
    aoi_src.at[aoi_ind, 'chip_id'] = 'chip_' + str(aoi_ind)
    
    # Subset potential images based on tile and date
    tile_mask = qtile == tname
    # Subset based on date
    date_mask1 = (qyear == yoi[0])
  #  diff = (qmonth-aoi_month)%12
   # date_mask2 = ((diff <= month_threshold[1]) & (diff >= month_threshold[0]))
    date_mask = date_mask1  #& date_mask2
    
    mask = tile_mask & date_mask
    candidate_dates = qdate[mask]
    candidate_tiles = qtile[mask]
    candidate_urls = qurl[mask]
    # Loop through images and check cloud cover
    for cd, ct, cu in zip(candidate_dates, candidate_tiles, candidate_urls):
       # print(cu)
        local_name = cu.split('/')[-1].replace("\n", "")
        print(local_name)
        if local_name != "HLS.S30.T14RNS.2020345.v1.4.hdf":
            continue
        print('valid')
        # Download a candidate file to check cloud cover
        # try:
        #     urlreq.urlretrieve(cu, filename=hdf_dir+local_name)
        # except:
        #     continue
    
        # Check cloud cover and spacial coverage
      #  try:
            # md = nasa_hls.get_metadata_from_hdf(hdf_dir+local_name, fields=['cloud_cover', 'spatial_coverage'])
            # if ((md["cloud_cover"] < cldfrac_threshold) and (md['spatial_coverage'] > spacecov_threshold)):
            
                # Convert to a geotiff            
                # nasa_hls.convert_hdf2tiffs(Path(hdf_dir+local_name), Path(tiff_dir))

            # Open the new geotiff
        date = datetime.strptime(cd, '%Y-%m-%d')
        doy = (date-datetime(date.year, 1, 1)).days+1
        fp = f'HLS.S30.T{ct}.{date.year}{doy:03d}.v1.4'
        src = rasterio.open(f'{tiff_dir}/{fp}__B04.tif')
        red = src.read(1)
        print(src)
        print(red)

        # Open NIR too to compute NDVI
        nir_src = rasterio.open(f'{tiff_dir}/{fp}__B08.tif')
        nir = nir_src.read(1)

        
        qa_src = rasterio.open(f'{tiff_dir}/{fp}__QA.tif')
        qa = qa_src.read(1)
        
        print(qa_src)
        print(qa)
        
        # Compute NDVI
        red = np.clip(red/15000, 0, 1)
        nir = np.clip(nir/15000, 0, 1)
        ndvi = np.clip((nir-red)/(nir+red), 0, 1)

        # Convert burn scar shape to proper geometry
        aoi_shape = aoi_src.to_crs(src.crs).iloc[aoi_ind]

        print(aoi_shape)
        # Build the mask
        out_image, out_transform = rasterio.mask.mask(src, [aoi_shape.geometry], crop=False)
        print(np.min(out_image), np.max(out_image))
        out_image = np.clip(out_image, 0, 1)
        print(np.min(out_image), np.max(out_image))

        out_meta = src.meta
        out_meta.update({'driver':'GTiff', 'height':out_image.shape[1],
            'width':out_image.shape[2], 'transform':out_transform})

        with rasterio.open(f'{mask_dir}/{fp}.mask.tif', 'w', **out_meta) as dest:
            dest.write(out_image)

        # Plot the original and mask
        fig, [ax1, ax2] = pp.subplots(ncols=2)
        ax1.imshow(ndvi, cmap='YlGn', vmin=0, vmax=1)
        ax2.imshow(np.squeeze(out_image), cmap='Greys')
        pp.savefig(f'{mask_dir}/{fp}.mask.jpg')
        pp.close()

        # Save in index file
#         fn_index = open(index_file, 'a')
#         fn_index.write(f'\n{aoi_row.Event_ID},{aoi_row.Incid_Name},{aoi_row.Ig_Date},{cd},{aoi_row.BurnBndAc},{ct},{fp}.mask.tif')
#         fn_index.close()

#         # Move onto the next burn scar
#         del src
#         del nir_src
        break
        # except:
        #     print('error')
        #     continue

geometry      POLYGON ((-98.46590447567304 28.48216163356960...
minx                                                 -98.492844
miny                                                  28.482162
maxx                                                 -98.465904
maxy                                                  28.491892
centroid_x                                           -98.479374
centroid_y                                            28.487027
tile                                                      14RNS
chip_id                                                  chip_0
Name: 0, dtype: object
        minx       miny       maxx       maxy
0 -98.492844  28.482162 -98.465904  28.491892
[69873.57854445 26546.24690202 22692.05707804 ... 15908.73972509
 12442.80659328  9532.39628432]
14RNS
HLS.S30.T14RNS.2020002.v1.4.hdf
HLS.S30.T14RNS.2020005.v1.4.hdf
HLS.S30.T14RNS.2020007.v1.4.hdf
HLS.S30.T14RNS.2020010.v1.4.hdf
HLS.S30.T14RNS.2020012.v1.4.hdf
HLS.S30.T14RNS.2020015.v1.4.hdf
HLS.S30.T14RNS

C:\Users\mcecil\AppData\Local\Temp\ipykernel_5544\1435243893.py:73: RuntimeWarning: invalid value encountered in true_divide
  ndvi = np.clip((nir-red)/(nir+red), 0, 1)


geometry      POLYGON ((552278.43611174 3150731.966929977, 5...
minx                                                 -98.492844
miny                                                  28.482162
maxx                                                 -98.465904
maxy                                                  28.491892
centroid_x                                           -98.479374
centroid_y                                            28.487027
tile                                                      14RNS
chip_id                                                  chip_0
Name: 0, dtype: object
-1000 2601
0 1
